In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
from matplotlib.gridspec import GridSpec
from astropy import constants as const
from mpl_toolkits.mplot3d import Axes3D
from collections import Counter

import warnings
warnings.filterwarnings("ignore")
import math

from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN

import scipy
from sklearn.cluster import KMeans
from scipy.stats import norm
from scipy.stats import skewnorm
from scipy.stats import multivariate_normal
from scipy.stats import bootstrap
from scipy import interpolate
from scipy.optimize import curve_fit

In [14]:
dataLI = pd.read_csv(r"C:\Users\silke\OneDrive\Skrivebord\Thesis\STPM_improved_210721.csv",delimiter=",")
dataEA = pd.read_csv(r"C:\Users\silke\OneDrive\Skrivebord\Thesis\EPA_8_jan_2024.txt",delimiter=",")
earth_like_rocky  = pd.read_csv(r"C:\Users\silke\OneDrive\Skrivebord\Thesis\Mass_Radius_Curves\Earth-like Rocky (32.5% Fe+67.5% MgSiO3).txt",delimiter = "\t")

planet_name_LI        = list(dataLI["Star"]+" "+dataLI["Planet"])
dataLI["planet_name"] = planet_name_LI
earth_Mass = const.M_earth.value*1000 #g
earth_Radius = const.R_earth.value*100
earth_Volume = 4/3 * math.pi * earth_Radius**3
earth_density = earth_Mass / earth_Volume

In [10]:
# Luque Impoved
planet_id_LI         = dataLI.ID
star_name_LI         = dataLI.Star
planet_letter_LI     = dataLI.Planet

radius_LI            = dataLI.R_Rterra            # [Earth Radius]
radius_unc_up_LI     = dataLI.euR_Rterra          # [Earth Radius] Upper Unc.
radius_unc_low_LI    = dataLI.edR_Rterra          # [Earth Radius] Lower Unc.

mass_LI              = dataLI.M_Mterra            # [Earth Mass] - Planet Mass or Mass*sin(i) [Earth Mass]
mass_unc_up_LI       = dataLI.euM_Mterra          # [Earth Mass] Upper Unc.
mass_unc_low_LI      = dataLI.edM_Mterra          # [Earth Mass] Lower Unc.

density_LI           = dataLI[dataLI.columns[32]]  # [g/cm**3]
density_unc_up_LI    = dataLI[dataLI.columns[33]]  # [g/cm**3] Planet Density Upper Unc. 
density_unc_low_LI   = dataLI[dataLI.columns[34]]  # [g/cm**3] Planet Density Lower Unc
star_eff_LI          = dataLI.Teff_K
pl_eq_LI             = dataLI.Teq_K                # [K] Equilibrium Temperature


planet_name_EA       = dataEA.pl_name
planet_id_EA         = list(range(0,len(planet_name_EA)))

radius_EA            = dataEA.pl_rade            # [Earth Radius]
period_EA            = dataEA.pl_orbper          # [days]
radius_unc_up_EA     = dataEA.pl_radeerr1        # [Earth Radius] Upper Unc.
radius_unc_low_EA    = dataEA.pl_radeerr2        # [Earth Radius] Lower Unc.
mass_EA              = dataEA.pl_bmasse          # [Earth Mass] - Planet Mass or Mass*sin(i) [Earth Mass]

mass_unc_up_EA       = dataEA.pl_bmasseerr1      # [Earth Mass] Upper Unc.
mass_unc_low_EA      = dataEA.pl_bmasseerr2      # [Earth Mass] Lower Unc.

density_EA           = dataEA.pl_dens            # [g/cm**3]
density_unc_up_EA    = dataEA.pl_denserr1        # [g/cm**3] Planet Density Upper Unc. 
density_unc_low_EA   = dataEA.pl_denserr2        # [g/cm**3] Planet Density Lower Unc. 
st_spectype_EA       = dataEA.st_spectype        # Spectral type of star
pl_eq_EA             = dataEA.pl_eqt             # [K] Equilibrium Temperature
st_eff_EA            = dataEA.st_teff            # Star effective temprature
st_eff_up_EA         = dataEA.st_tefferr1        # Star upper err effective temprature
st_eff_low_EA        = dataEA.st_tefferr2        # Star lower err effective temprature
disc_year_EA         = dataEA.disc_year


In [11]:
planets = ["K2-146 b",'TRAPPIST-1 h', 'TRAPPIST-1 d', 'TRAPPIST-1 e', 'TRAPPIST-1 f', 'TRAPPIST-1 g', 'TRAPPIST-1 c', 'TRAPPIST-1 b', 'GJ 1132 b', 'TOI-270 b', 'GJ 3053 c', 'GJ 1252 b', 'GJ 357 b', 'LTT 3780 b', 'LHS 1478 b', 'GJ 486 b', 'GJ 3473 b', 'CD-60 8051 b', 'L 98-59 c', 'L 98-59 d', 'GJ 3053 b', 'TOI-270 d', 'TOI-776 b', 'TOI-1634 b', 'TOI-1685 b', 'TOI-1235 b', 'K2-146 c', 'LTT 3780 c', 'TOI-270 c', 'K2-18 b', 'TOI-269 b', 'GJ 1214 b', 'K2-25 b', 'TOI-1231 b']
planet1      = []; planet_i1       = []; planet_eq1   = []
disc_year1   = []; st_eff1         = []; st_type1     = []
radius1      = []; radius_up1      = []; radius_low1  = []
mass1        = []; mass_up1        = []; mass_low1    = []
density1     = []; density_up1     = []; density_low1 = []

for i in range(0,len(planet_name_LI)):
    if planet_name_LI[i] in planets:
        planet1.append(planet_name_LI[i]); planet_eq1.append(pl_eq_LI[i]);          st_eff1.append(star_eff_LI[i]);
        radius1.append(radius_LI[i]);      radius_up1.append(radius_unc_up_LI[i]);  radius_low1.append(radius_unc_low_LI[i]);
        mass1.append(mass_LI[i]);          mass_up1.append(mass_unc_up_LI[i]);      mass_low1.append(mass_unc_low_LI[i]);
        density1.append(density_LI[i]);    density_up1.append(density_unc_up_LI[i]); density_low1.append(density_unc_low_LI[i]);

In [12]:
planet_EA_unique = list(set(planet_name_EA))
other_names = ["LHS 1140 b",'LHS 1140 c' ,'L 168-9 b', 'L 168-9 c', "GJ 3053 b","GJ 3053 c"]

radius_4_crit     = []
mass_err_crit     = []
radius_err_crit   = []
stellar_type_crit = []
st_eff_crit       = []
density_crit      = []
for i in range(0,len(planet_name_EA)):
    if planet_name_EA[i] not in planet_name_LI and planet_name_EA[i] not in other_names and planet_name_EA[i]:
        if planet_name_EA[i] not in radius_4_crit and radius_EA[i]<4:
            radius_4_crit.append(planet_name_EA[i])
        if planet_name_EA[i] not in mass_err_crit and ((mass_unc_up_EA[i] + mass_unc_up_EA[i])*0.5) / mass_EA[i] < 0.50:
            mass_err_crit.append(planet_name_EA[i])    
        if planet_name_EA[i] not in radius_err_crit and ((radius_unc_up_EA[i] + radius_unc_up_EA[i])*0.5) / radius_EA[i] < 0.50:
            radius_err_crit.append(planet_name_EA[i])   
        if planet_name_EA[i] not in stellar_type_crit and "M" in str(st_spectype_EA[i]):
            stellar_type_crit.append(planet_name_EA[i])   
        if planet_name_EA[i] not in density_crit and True != math.isnan(density_unc_up_EA[i]):
            density_crit.append(planet_name_EA[i])   

list1 = radius_4_crit;   list2 = mass_err_crit; 
list3 = radius_err_crit; list4 = stellar_type_crit; 
list5 = density_crit

common_elements = np.sort(list(set(list1) & set(list2) & set(list3) & set(list4) & set(list5)))

planet2      = []; planet_i2       = []; planet_eq2   = []
disc_year2   = []; st_eff2         = []
radius2      = []; radius_up2      = []; radius_low2  = []
mass2        = []; mass_up2        = []; mass_low2    = []
density2     = []; density_up2     = []; density_low2 = []

for j in range(0,len(common_elements)):
    target_string = str(common_elements[j])
    indexs = [index for index, value in enumerate(planet_name_EA) if value == target_string]
    i = indexs[0]
    if st_eff_EA[i] < 4000:
        st_eff2.append(st_eff_EA[i])
        planet_eq2.append(pl_eq_EA[i]); planet2.append(planet_name_EA[i]); disc_year2.append(disc_year_EA[i])
        radius2.append(dataEA.pl_rade[i]); radius_up2.append(dataEA.pl_radeerr1[i]); radius_low2.append(dataEA.pl_radeerr2[i])
        density2.append(dataEA.pl_dens[i]); density_up2.append(dataEA.pl_denserr1[i]); density_low2.append(dataEA.pl_denserr2[i])
        mass2.append(dataEA.pl_bmasse[i]); mass_up2.append(dataEA.pl_bmasseerr1[i]);  mass_low2.append(dataEA.pl_bmasseerr2[i])


In [15]:
earth_like_rocky_mass_g        = earth_Mass*np.array(earth_like_rocky["Mass"])
earth_like_rocky_radius_cm     = earth_Radius*np.array(earth_like_rocky["Radius"])
earth_like_rocky_density_g_cm3 = (earth_like_rocky_mass_g) / ((4/3)*(np.pi)*(np.power(earth_like_rocky_radius_cm,3))   ) 

def rocky_density(mass):
    x_points  = earth_like_rocky["Mass"]
    y_points  = earth_like_rocky_density_g_cm3
    tck       = interpolate.splrep(x_points, y_points)
    return interpolate.splev(mass, tck).tolist()

def rocky_density2(mass):
    x_points  = earth_like_rocky["Mass"]
    y_points  = earth_like_rocky_density_g_cm3/earth_like_rocky_density_g_cm3
    tck       = interpolate.splrep(x_points, y_points)
    return interpolate.splev(mass, tck).tolist()

def h20model_density(mass):
    x_points  = H20_50_500K["Mass"]
    y_points  = H20_50_500K_density_g_cm3/rocky_density(H20_50_500K["Mass"])
    tck       = interpolate.splrep(x_points, y_points)
    return interpolate.splev(mass, tck).tolist()

mass    = np.concatenate((mass1, mass2));       mass_up     = np.concatenate((mass_up1, mass_up2));       mass_low     = np.concatenate((mass_low1, mass_low2));
radius  = np.concatenate((radius1, radius2));   radius_up   = np.concatenate((radius_up1, radius_up2));   radius_low   = np.concatenate((radius_low1, radius_low2));
density = np.concatenate((np.array(density1) / rocky_density(mass1), np.array(density2)/ rocky_density(mass2))); density_up  = np.concatenate((np.array(density_up1)/ rocky_density(mass1), np.array(density_up2)/ rocky_density(mass2))); density_low  = np.concatenate((np.array(density_low1)/ rocky_density(mass1),np.array(density_low2)/ rocky_density(mass2)));

planet_temp = np.concatenate((planet_eq1, planet_eq2));
planet_name = np.concatenate((planet1,    planet2));
star_eff    = np.concatenate((st_eff1,    st_eff2));
disc_year   = np.concatenate((disc_year1,    disc_year2));

mass_err    = [np.abs(mass_low),mass_up]
radius_err  = [np.abs(radius_low),radius_up]
density_err = [np.abs(density_low),density_up]

mass10      = np.log10(mass)
mass10_up   = np.log10(mass + mass_up) - np.log10(mass)
mass10_low  = np.log10(mass + np.abs(mass_low)) - np.log10(mass)
mass10_err  = [mass10_low,mass10_up]

model_rocky_density = np.array(rocky_density(np.array(earth_like_rocky["Mass"])))
model_rocky_mass    = np.array(earth_like_rocky["Mass"])
model_rocky10_mass  = np.array(np.log10(np.array(earth_like_rocky["Mass"])))
model_rocky_radius  = np.array(earth_like_rocky["Radius"])

## Predictions with Errors

In [32]:
All_with_errors = pd.read_csv(r"C:\Users\silke\ExoMDN\more_examples\Planet_param_with_errors.txt",delimiter = "\t")

In [34]:
All_with_errors

,log_d_mantle_core,log_d_water_core,log_d_atmosphere_core,log_m_mantle_core,log_m_water_core,log_m_atmosphere_core,prediction,mantle_rf,water_rf,atmosphere_rf,core_rf,mantle_mf,water_mf,atmosphere_mf,core_mf,Planet Name
0,-1.063421,0.791839,-1.854881,-2.392138,1.589504,-10.393211,GJ 3053 b,0.093086,0.595130,0.042185,0.269600,0.015257,0.817870,5.112780e-06,0.166867,GJ 3053 b
1,-1.298983,0.541428,-1.397346,-1.547932,1.287544,-7.106729,GJ 3053 b,0.084239,0.530631,0.076347,0.308783,0.043967,0.749140,1.694246e-04,0.206724,GJ 3053 b
2,-3.198655,0.661593,-1.579188,-1.083121,1.429524,-9.791395,GJ 3053 b,0.012816,0.608470,0.064726,0.313988,0.061381,0.757295,1.014102e-05,0.181314,GJ 3053 b
3,-1.288198,0.288596,-1.074861,-1.016454,0.803274,-5.061243,GJ 3053 b,0.093428,0.452135,0.115645,0.338792,0.100492,0.620052,1.759951e-03,0.277697,GJ 3053 b
4,-0.484744,1.020920,-2.201874,0.217573,2.302603,-12.435337,GJ 3053 b,0.136790,0.616532,0.024565,0.222114,0.101530,0.816792,3.247173e-07,0.081678,GJ 3053 b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18736295,-1.590269,-0.498330,-2.429444,-1.431920,-0.535119,-13.798254,4261,0.107329,0.319844,0.046373,0.526454,0.130916,0.320973,5.576509e-07,0.548110,TOI-2136 b
18736296,-0.931763,-0.508256,-2.984514,-0.420335,-0.536075,-14.588398,4261,0.192505,0.294015,0.024714,0.488766,0.292982,0.260961,2.059346e-07,0.446057,TOI-2136 b
18736297,-0.732047,-2.531141,-1.280902,-0.261914,-2.097840,-5.205078,4261,0.261616,0.043284,0.151112,0.543987,0.405513,0.064665,2.892119e-03,0.526930,TOI-2136 b
18736298,-1.489463,-0.751256,-2.032941,-1.151158,-0.864725,-9.575195,4261,0.123341,0.258051,0.071627,0.546981,0.182025,0.242397,3.995941e-05,0.575537,TOI-2136 b


In [46]:
planet_rf = pd.DataFrame()
planet_rf['Planet Name'] = All_with_errors['Planet Name']
planet_rf['core_rf'] = All_with_errors['core_rf']
planet_rf['mantle_rf'] = All_with_errors['mantle_rf']
planet_rf['water_rf'] = All_with_errors['water_rf']
planet_rf['atmosphere_rf'] = All_with_errors['atmosphere_rf']
unique_entries = planet_rf['Planet Name'].unique()


In [48]:
random_row = planet_rf.loc[planet_rf['Planet Name'] == 'TOI-2136 b'].sample()

In [56]:
random_row

,Planet Name,core_rf,mantle_rf,water_rf,atmosphere_rf
18613323,TOI-2136 b,0.542706,0.371053,0.004387,0.081854


In [62]:
random_row = planet_rf.loc[planet_rf['Planet Name'] == unique_entries[10]].sample()

In [67]:
random_dataframe = pd.DataFrame()
for i in range(0,50):
    random_row = planet_rf.loc[planet_rf['Planet Name'] == unique_entries[i]].sample()
    random_dataframe = pd.concat([random_dataframe, random_row], ignore_index=True)


In [68]:
random_dataframe

,Planet Name,core_rf,mantle_rf,water_rf,atmosphere_rf
0,GJ 3053 b,0.172891,1.699743e-01,5.836778e-01,7.345676e-02
1,GJ 3053 c,0.153192,5.101765e-02,2.925292e-02,7.665378e-01
2,LHS 1478 b,0.603842,7.214867e-02,1.894641e-01,1.345449e-01
3,TOI-1634 b,0.999931,3.457997e-09,2.980160e-06,6.562113e-05
4,GJ 3473 b,0.238628,1.100792e-01,2.298300e-01,4.214632e-01
5,L 98-59 c,0.997997,1.145488e-03,5.907957e-05,7.985983e-04
6,L 98-59 d,0.206894,1.181535e-02,1.361705e-01,6.451200e-01
7,K2-146 b,0.213209,6.204053e-02,1.699085e-01,5.548416e-01
8,K2-146 c,0.293225,3.644813e-02,4.868641e-01,1.834630e-01
9,GJ 357 b,0.206183,7.318141e-02,1.729919e-01,5.476434e-01


In [94]:
X = np.column_stack((random_dataframe['core_rf'], random_dataframe['mantle_rf'], random_dataframe['water_rf'], random_dataframe['atmosphere_rf']))
kmeans = KMeans(n_clusters=3,n_init = 10)
labels = kmeans.fit_predict(X)
scatter(filtered_mass, filtered_density , c=labels, cmap=custom_cmap,s = 100)

AttributeError: 'NoneType' object has no attribute 'split'

In [93]:
labels

NameError: name 'labels' is not defined

In [79]:
# Sample data
main_list = unique_entries
names = planet_name


# Zip the three lists together
planet_data = list(zip(planet_name, mass10, density))

# Filter the data based on the main list
filtered_data = [planet for planet in planet_data if planet[0] in main_list]

# Unzip the filtered data
filtered_names, filtered_mass, filtered_density = zip(*filtered_data)

# Print the result
print("Filtered Names:", filtered_names)
print("Filtered Mass:", filtered_mass)
print("Filtered density:", filtered_density)


Filtered Names: ('GJ 3053 b', 'GJ 3053 c', 'LHS 1478 b', 'TOI-1634 b', 'GJ 3473 b', 'L 98-59 c', 'L 98-59 d', 'K2-146 b', 'K2-146 c', 'GJ 357 b', 'TOI-1235 b', 'GJ 1132 b', 'LTT 3780 b', 'LTT 3780 c', 'K2-18 b', 'TOI-776 b', 'GJ 486 b', 'GJ 1214 b', 'TRAPPIST-1 b', 'TRAPPIST-1 c', 'TRAPPIST-1 d', 'TRAPPIST-1 e', 'TRAPPIST-1 f', 'TRAPPIST-1 g', 'TRAPPIST-1 h', 'CD-60 8051 b', 'TOI-270 b', 'TOI-270 c', 'TOI-270 d', 'K2-25 b', 'TOI-269 b', 'TOI-1231 b', 'AU Mic c', 'GJ 3090 b', 'GJ 3929 b', 'GJ 806 b', 'HD 260655 b', 'HD 260655 c', 'Kepler-26 b', 'Kepler-26 c', 'L 98-59 b', 'LTT 1445 A b', 'LTT 1445 A c', 'TOI-1201 b', 'TOI-1452 b', 'TOI-1470 b', 'TOI-1470 c', 'TOI-1695 b', 'TOI-1801 b', 'TOI-2136 b')
Filtered Mass: (0.8048206787211624, 0.24551266781414982, 0.367355921026019, 0.8790958795000727, 0.26951294421791633, 0.38381536598043126, 0.36361197989214433, 0.7611758131557314, 0.8744818176994665, 0.2648178230095365, 0.8254261177678232, 0.22010808804005508, 0.39269695325966575, 0.846337112

In [85]:
# Combine the lists to create the 4D dataset
X = np.column_stack((random_dataframe['Core'], random_dataframe['Mantle'], random_dataframe['Water'], random_dataframe['Atmosphere']))

# Apply K-means clustering with 3 clusters (adjust as needed)
kmeans = KMeans(n_clusters=2,n_init = 10)
labels = kmeans.fit_predict(X)

scatter(filtered_mass, filtered_density, c=labels, cmap=custom_cmap,s = 100)

# Adjust layout for better spacing
plt.tight_layout(w_pad=5.0)
plt.show()

KeyError: 'Core'